# BTS 모델(자체 개발 모델)의 성능 측정

In [17]:
import sys
sys.path.append('..')
from Utils import content_based_filtering
import pandas as pd
from tqdm import tqdm
import random

인기있는 영화와 MovieLens 데이터에서 겹치는 영화를 찾아낸다.  
-> 인기있는 영화의 interaction 데이터를 얻기위해

In [33]:
# # specify the start and end row indices
# start_index = 0
# end_index = 80

# # group the DataFrame by the 'userId' column
# grouped = interactions_df.groupby("userId")

# # extract the specified rows for each group into a list of DataFrames
# result = [group.iloc[start_index:end_index] for _, group in grouped]

# # concatenate the list of DataFrames into a single DataFrame
# train_df = pd.concat(result)

# # specify the start and end row indices
# start_index = 80
# end_index = 140 #100

# # # group the DataFrame by the 'userId' column
# # grouped = interactions_df.groupby("userId")

# # extract the specified rows for each group into a list of DataFrames
# result = [group.iloc[start_index:] for _, group in grouped]

# # concatenate the list of DataFrames into a single DataFrame
# test_df = pd.concat(result)


In [9]:
mv_df = pd.read_pickle('../Utils/Pickle/230130_Popular_movie_1192_cwj.pickle')
ml_df = pd.read_csv('../Data/EDA/ml-25m/ratings.csv')

merged_df = pd.merge(ml_df,mv_df,how='inner',on='movieId')

merged_df = merged_df.drop(['rating_x', 'timestamp', 'Contents', 'ko_title',
       'contents_year', 'country', 'ko_genre', 'ko_plot', 'rating_y', 'npop',
       'genres', 'plot', 'directors', 'actors'],axis=1)

In [14]:
merged_df.userId.nunique()

user_ids = sorted(set(merged_df.userId))
len(user_ids)

random_user_ids = random.sample(user_ids,len(user_ids)//10)
len(random_user_ids)

byuser_item = merged_df[merged_df.userId.isin(random_user_ids)].groupby('userId').movieId.count().reset_index()
byuser_item

byuser_item.movieId.sort_values()

161859

In [52]:
mask = merged_df.userId.isin(merged_df.userId.value_counts()[merged_df.userId.value_counts() > 1].index)
interactions_df = merged_df[mask]

interactions_df = interactions_df.sort_values('userId')

In [53]:
interactions_df.userId.nunique()
interactions_df.shape

(6346505, 2)

In [58]:
interactions_df.groupby('userId').movieId.count().reset_index().sort_values('movieId')

,userId,movieId
100182,101787,2
114577,116422,2
155222,157779,2
54722,55631,2
138638,140920,2
...,...,...
121739,123711,694
30144,30643,723
109293,111038,742
56618,57548,775


In [82]:
user_ids = list(set(interactions_df.userId))

In [95]:
random_user_ids = random.sample(user_ids,600)

In [96]:
len(random_user_ids)

600

HitRate@10을 구한다.  
80개의 interaction을 입력하고 10개의 추천리스트를 만든다.  
추천 리스트에서 입력으로 넣지 않은 유저-영화 상호작용한 데이터가 있으면 Hit이다.

In [85]:
valid_df = interactions_df[interactions_df.userId.isin(random_user_ids)]
valid_df

,userId,movieId
400005,122,318
5525480,122,339
1456889,122,50
4447135,122,539
59,122,296
...,...,...
79602,162429,296
4550721,162429,597
4289308,162429,377
481364,162429,318


In [97]:
df = valid_df.groupby("userId").movieId.apply(list).reset_index()

In [98]:
def divide_list(lst):
    half = len(lst)//2
    return lst[:half], lst[half:]

In [99]:
cnt=0
for l in tqdm(df.itertuples(), total=df.shape[0]):
    train, test = divide_list(l.movieId)
    reccomend_list = [i[0] for i in content_based_filtering(train,10)]
#     print(reccomend_list, test)
    if set(reccomend_list).intersection(test):
        cnt+=1
    else:
        pass
print(cnt)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [34:29<00:00,  4.14s/it]

19


In [101]:
print(len(test_df))
print(len(test_dict))
cnt/len(test_dict) # HitRate@10

1462410
17772


0.0010690974566734189